# Phase 4: Business Simulation — ROI of Uplift Targeting

## Uplift Modeling Project — "The Persuadable Hunter"

This notebook answers the key business question:

> **"If I can only email X% of customers, who should I pick?"**

We compare three strategies:
1. **Spray & Pray** — Email everyone
2. **Random Targeting** — Email a random X%
3. **Uplift Targeting** — Email top X% by uplift score (our T-Learner)

---

### Cost/Profit Parameters

| Parameter | Value | Rationale |
|-----------|-------|----------|
| **Email Cost** | $0.10 | Digital campaign cost (ESP, design, opportunity cost) |
| **Profit Margin** | $25.00 | Net profit per conversion |

---

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import trapezoid

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

import warnings
warnings.filterwarnings('ignore')

# Business parameters
EMAIL_COST = 0.10  # Realistic digital campaign cost
PROFIT_MARGIN = 25.00

print(f"Business Parameters:")
print(f"  Email Cost: ${EMAIL_COST:.2f}")
print(f"  Profit Margin: ${PROFIT_MARGIN:.2f}")
print(f"  Break-even conversion rate: {EMAIL_COST/PROFIT_MARGIN*100:.2f}%")

In [ ]:
# Load test results with uplift scores
results_df = pd.read_csv('../data/test_results_with_uplift.csv')

print(f"Test Results: {len(results_df):,} customers")
print(f"Treated: {results_df['treatment'].sum():,}")
print(f"Control: {(1-results_df['treatment']).sum():,}")
print(f"Conversions: {results_df['y_true'].sum():,} ({results_df['y_true'].mean()*100:.2f}%)")

---

## 2. Strategy Comparison Framework

### How We Calculate ROI (Counterfactual Projection)

For any targeting strategy at X% of population:

```
Customers Targeted = X% × Total Customers
Cost = Customers Targeted × Email Cost

# Estimate uplift rate from RCT data
Uplift Rate = Treatment Conversion Rate - Control Conversion Rate

# Project to ALL targeted customers (not just historically treated)
Incremental Conversions = Uplift Rate × Customers Targeted
Revenue = Incremental Conversions × Profit Margin

Profit = Revenue - Cost
```

### Key Insight

We estimate the **uplift rate** from the RCT sample, then project it to the **full** target population. This is the correct counterfactual: "If we email ALL selected customers, how many incremental conversions do we expect?"

In [ ]:
def calculate_profit_curve(results_df, strategy='uplift', n_points=100):
    """
    Calculate profit at different targeting percentages.
    
    FIXED (Codex Review): Properly projects uplift rate to ALL targeted customers,
    not just the historically treated subset.
    """
    y_true = results_df['y_true'].values
    treatment = results_df['treatment'].values
    uplift_scores = results_df['uplift_score'].values
    
    n = len(results_df)
    
    # Sort by strategy
    if strategy == 'uplift':
        sorted_idx = np.argsort(uplift_scores)[::-1]  # Highest uplift first
    elif strategy == 'random':
        np.random.seed(42)
        sorted_idx = np.random.permutation(n)
    else:
        raise ValueError(f"Unknown strategy: {strategy}")
    
    y_sorted = y_true[sorted_idx]
    t_sorted = treatment[sorted_idx]
    
    # Calculate at each targeting percentage
    percentages = []
    profits = []
    revenues = []
    costs = []
    incremental_conversions = []
    
    step = max(1, n // n_points)
    
    for i in range(1, n + 1, step):
        pct = i / n
        
        # ALL targeted customers pay the cost
        cost = i * EMAIL_COST
        
        # Get conversions in this subset
        y_subset = y_sorted[:i]
        t_subset = t_sorted[:i]
        
        n_t_subset = t_subset.sum()
        n_c_subset = i - n_t_subset
        
        conversions_t = (y_subset * t_subset).sum()
        conversions_c = (y_subset * (1 - t_subset)).sum()
        
        # FIX (Codex): Estimate uplift RATE, then project to ALL targeted customers
        if n_t_subset > 0 and n_c_subset > 0:
            treated_rate = conversions_t / n_t_subset
            control_rate = conversions_c / n_c_subset
            uplift_rate = treated_rate - control_rate
            incr_conv = uplift_rate * i  # Project to ALL targeted customers
        else:
            incr_conv = 0
        
        revenue = incr_conv * PROFIT_MARGIN
        profit = revenue - cost
        
        percentages.append(pct)
        profits.append(profit)
        revenues.append(revenue)
        costs.append(cost)
        incremental_conversions.append(incr_conv)
    
    # Ensure endpoint
    if percentages[-1] != 1.0:
        i = n
        cost = i * EMAIL_COST
        
        n_treated = treatment.sum()
        n_control = n - n_treated
        total_t_conv = (y_true * treatment).sum()
        total_c_conv = (y_true * (1 - treatment)).sum()
        
        treated_rate = total_t_conv / n_treated if n_treated > 0 else 0
        control_rate = total_c_conv / n_control if n_control > 0 else 0
        uplift_rate = treated_rate - control_rate
        incr_conv = uplift_rate * i
        
        revenue = incr_conv * PROFIT_MARGIN
        profit = revenue - cost
        
        percentages.append(1.0)
        profits.append(profit)
        revenues.append(revenue)
        costs.append(cost)
        incremental_conversions.append(incr_conv)
    
    return {
        'percentages': np.array(percentages),
        'profits': np.array(profits),
        'revenues': np.array(revenues),
        'costs': np.array(costs),
        'incremental_conversions': np.array(incremental_conversions)
    }

print("Profit curve function defined (with Codex counterfactual fix)!")

In [ ]:
# Calculate profit curves for both strategies
uplift_curve = calculate_profit_curve(results_df, strategy='uplift')
random_curve = calculate_profit_curve(results_df, strategy='random')

print("Profit curves calculated!")
print(f"\nUplift Strategy at 100%: ${uplift_curve['profits'][-1]:.2f}")
print(f"Random Strategy at 100%: ${random_curve['profits'][-1]:.2f}")

---

## 3. Spray & Pray Baseline

What if we just email **everyone**?

In [ ]:
# Spray & Pray = 100% targeting
spray_and_pray_profit = uplift_curve['profits'][-1]
spray_and_pray_cost = uplift_curve['costs'][-1]
spray_and_pray_revenue = uplift_curve['revenues'][-1]
spray_and_pray_incr_conv = uplift_curve['incremental_conversions'][-1]

print("SPRAY & PRAY ANALYSIS (Email Everyone)")
print("=" * 60)
print(f"Customers Emailed:       {len(results_df):,}")
print(f"Email Cost:              ${spray_and_pray_cost:,.2f}")
print(f"Incremental Conversions: {spray_and_pray_incr_conv:.1f}")
print(f"Revenue:                 ${spray_and_pray_revenue:,.2f}")
print(f"Profit:                  ${spray_and_pray_profit:,.2f}")
print(f"ROI:                     {(spray_and_pray_revenue/spray_and_pray_cost - 1)*100:.1f}%")

---

## 4. Profit Curve: T-Learner vs Random

In [ ]:
# Plot Profit Curve
fig, ax = plt.subplots(figsize=(14, 8))

# Uplift targeting
ax.plot(uplift_curve['percentages'] * 100, uplift_curve['profits'], 
        'b-', linewidth=2.5, label='T-Learner (Uplift Targeting)')

# Random targeting
ax.plot(random_curve['percentages'] * 100, random_curve['profits'], 
        'r--', linewidth=2, label='Random Targeting')

# Spray & Pray reference line
ax.axhline(y=spray_and_pray_profit, color='gray', linestyle=':', linewidth=1.5, 
           label=f'Spray & Pray (${spray_and_pray_profit:,.0f})')

# Break-even line
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

# Find optimal targeting percentage for uplift
optimal_idx = np.argmax(uplift_curve['profits'])
optimal_pct = uplift_curve['percentages'][optimal_idx] * 100
optimal_profit = uplift_curve['profits'][optimal_idx]

ax.scatter([optimal_pct], [optimal_profit], color='green', s=150, zorder=5, marker='*')
ax.annotate(f'OPTIMAL: {optimal_pct:.0f}%\nProfit: ${optimal_profit:,.0f}', 
            xy=(optimal_pct, optimal_profit),
            xytext=(optimal_pct + 10, optimal_profit + 100),
            fontsize=11, fontweight='bold', color='green',
            arrowprops=dict(arrowstyle='->', color='green', lw=2))

# Fill area where uplift beats random
ax.fill_between(uplift_curve['percentages'] * 100, 
                random_curve['profits'], 
                uplift_curve['profits'],
                where=(uplift_curve['profits'] > random_curve['profits']),
                alpha=0.2, color='green', label='Uplift Advantage')

ax.set_xlabel('% of Customers Targeted', fontsize=14)
ax.set_ylabel('Profit ($)', fontsize=14)
ax.set_title('ROI Curve: T-Learner vs Random vs Spray & Pray\n(Cost: $0.10/email, Margin: $25.00/conversion)', 
             fontsize=16, fontweight='bold')
ax.legend(loc='lower right', fontsize=11)
ax.grid(True, alpha=0.3)

# Format y-axis as currency
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

plt.tight_layout()
plt.savefig('../data/roi_curve.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Summary table at key targeting percentages
print("\nPROFIT COMPARISON BY TARGETING PERCENTAGE")
print("=" * 80)
print(f"{'% Targeted':<12} {'Uplift Profit':<18} {'Random Profit':<18} {'Advantage':<15}")
print("-" * 80)

for target_pct in [0.1, 0.2, 0.3, 0.5, 0.7, 1.0]:
    # Find closest index
    uplift_idx = np.argmin(np.abs(uplift_curve['percentages'] - target_pct))
    random_idx = np.argmin(np.abs(random_curve['percentages'] - target_pct))
    
    uplift_profit = uplift_curve['profits'][uplift_idx]
    random_profit = random_curve['profits'][random_idx]
    advantage = uplift_profit - random_profit
    
    print(f"{target_pct*100:>6.0f}%       ${uplift_profit:>12,.2f}     ${random_profit:>12,.2f}     ${advantage:>10,.2f}")

print("-" * 80)
print(f"\nOptimal Targeting: {optimal_pct:.0f}% with profit of ${optimal_profit:,.2f}")

---

## 5. Decile 9 Profitability Analysis — The 'Loyalty Tax'

Gemini identified that Decile 9 contains "Sure Things" — high baseline converters that the model mistakenly ranked highly.

**Hypothesis**: Decile 8 (true Persuadables) is profitable, but Decile 9 (Sure Things) is unprofitable.

In [ ]:
# Create decile analysis with FIXED cost calculation (Codex review)
results_df['decile'] = pd.qcut(results_df['uplift_score'], q=10, labels=False, duplicates='drop') + 1

decile_analysis = []

for decile in range(1, 11):
    subset = results_df[results_df['decile'] == decile]
    
    treated = subset[subset['treatment'] == 1]
    control = subset[subset['treatment'] == 0]
    
    n_treated = len(treated)
    n_control = len(control)
    n_total = len(subset)  # Everyone in decile
    
    if n_treated > 0 and n_control > 0:
        # Calculate conversion rates
        treated_conv_rate = treated['y_true'].mean()
        control_conv_rate = control['y_true'].mean()
        uplift_rate = treated_conv_rate - control_conv_rate
        
        # FIX (Codex): Project to ALL customers in decile, not just n_treated
        incremental_conv = uplift_rate * n_total
        
        # FIX (Codex): Cost is for ALL customers in decile
        cost = n_total * EMAIL_COST
        
        revenue = incremental_conv * PROFIT_MARGIN
        profit = revenue - cost
        roi = (revenue / cost - 1) * 100 if cost > 0 else 0
        
        decile_analysis.append({
            'decile': decile,
            'n_customers': n_total,
            'treated_conv_rate': treated_conv_rate * 100,
            'control_conv_rate': control_conv_rate * 100,
            'observed_lift': uplift_rate * 100,
            'incremental_conv': incremental_conv,
            'cost': cost,
            'revenue': revenue,
            'profit': profit,
            'roi': roi
        })

decile_df = pd.DataFrame(decile_analysis)
print("Decile Profitability Analysis (with Codex counterfactual fix):")
print("=" * 110)
print(decile_df[['decile', 'n_customers', 'treated_conv_rate', 'control_conv_rate', 
                  'observed_lift', 'incremental_conv', 'profit', 'roi']].round(2).to_string(index=False))

In [ ]:
# Visualize Decile Profitability
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Left: Profit by Decile — Color-coded for profitability
ax = axes[0]

# Color scheme: Green=profitable, Red=Decile 9, Gray=unprofitable
colors = []
for d, p in zip(decile_df['decile'], decile_df['profit']):
    if d == 9:
        colors.append('#e74c3c')  # Red for Decile 9
    elif p > 0:
        colors.append('#27ae60')  # Green for profitable
    else:
        colors.append('#95a5a6')  # Gray for unprofitable

bars = ax.bar(decile_df['decile'], decile_df['profit'], color=colors, alpha=0.8, edgecolor='white', linewidth=1.5)
ax.axhline(y=0, color='black', linestyle='-', linewidth=1)

# Add value labels
for bar, profit in zip(bars, decile_df['profit']):
    height = bar.get_height()
    label_y = height + 10 if height >= 0 else height - 25
    ax.text(bar.get_x() + bar.get_width()/2, label_y, f'${profit:.0f}',
            ha='center', va='bottom' if height >= 0 else 'top', fontsize=9, fontweight='bold')

ax.set_xlabel('Uplift Decile (1=Lowest, 10=Highest)', fontsize=12)
ax.set_ylabel('Profit ($)', fontsize=12)
ax.set_title('Profit by Decile — The Decile 9 "Loyalty Tax"', fontsize=14, fontweight='bold')
ax.set_xticks(decile_df['decile'])

# Annotate Decile 9
d9_profit = decile_df[decile_df['decile'] == 9]['profit'].values[0]
d9_status = 'PROFITABLE' if d9_profit > 0 else 'UNPROFITABLE'
d9_color = '#27ae60' if d9_profit > 0 else '#e74c3c'
ax.annotate(f'Decile 9: {d9_status}\n${d9_profit:.0f}', 
            xy=(9, d9_profit),
            xytext=(7, d9_profit + 100 if d9_profit > 0 else d9_profit - 100),
            arrowprops=dict(arrowstyle='->', color=d9_color, lw=2),
            fontsize=10, color=d9_color, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='#fff3cd' if d9_profit > 0 else '#f8d7da', alpha=0.8))

# Annotate Decile 8 for comparison
d8_profit = decile_df[decile_df['decile'] == 8]['profit'].values[0]
if d8_profit > 0:
    ax.annotate(f'Decile 8: PROFITABLE\n${d8_profit:.0f}', 
                xy=(8, d8_profit),
                xytext=(5.5, d8_profit + 150),
                arrowprops=dict(arrowstyle='->', color='#27ae60', lw=2),
                fontsize=10, color='#27ae60', fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='#d4edda', alpha=0.8))

# Right: Cumulative Profit (targeting from Decile 10 down)
ax = axes[1]
decile_df_sorted = decile_df.sort_values('decile', ascending=False)
decile_df_sorted['cumulative_profit'] = decile_df_sorted['profit'].cumsum()
decile_df_sorted['cumulative_pct'] = decile_df_sorted['n_customers'].cumsum() / decile_df_sorted['n_customers'].sum() * 100

ax.plot(decile_df_sorted['cumulative_pct'], decile_df_sorted['cumulative_profit'], 
        'b-o', linewidth=2, markersize=8)

# Mark each decile
for _, row in decile_df_sorted.iterrows():
    d = int(row['decile'])
    color = 'red' if d == 9 else 'blue'
    fontweight = 'bold' if d == 9 else 'normal'
    ax.annotate(f'D{d}', xy=(row['cumulative_pct'], row['cumulative_profit']),
                xytext=(row['cumulative_pct'] + 3, row['cumulative_profit']),
                fontsize=9, color=color, fontweight=fontweight)

ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax.set_xlabel('% of Customers Targeted (High to Low Uplift)', fontsize=12)
ax.set_ylabel('Cumulative Profit ($)', fontsize=12)
ax.set_title('Cumulative Profit — Impact of Adding Decile 9', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/decile_profitability.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Decile 8 vs Decile 9 Comparison — The Key Insight
d9 = decile_df[decile_df['decile'] == 9].iloc[0]
d8 = decile_df[decile_df['decile'] == 8].iloc[0]
d10 = decile_df[decile_df['decile'] == 10].iloc[0]

print("DECILE 8 vs DECILE 9 — The 'Loyalty Tax' Hypothesis")
print("=" * 70)
print(f"""
THE HYPOTHESIS:
  Decile 8 contains TRUE Persuadables (high lift, moderate baseline)
  Decile 9 contains SURE THINGS (low lift, high baseline)
  
THE DATA:
  ┌──────────────────────────────────────────────────────────────┐
  │ Metric            │ Decile 8      │ Decile 9      │ Winner  │
  ├──────────────────────────────────────────────────────────────┤
  │ Control Rate      │ {d8['control_conv_rate']:>6.2f}%       │ {d9['control_conv_rate']:>6.2f}%       │ {'D8 ✓' if d8['control_conv_rate'] < d9['control_conv_rate'] else 'D9'}    │
  │ Observed Lift     │ {d8['observed_lift']:>+6.2f}pp      │ {d9['observed_lift']:>+6.2f}pp      │ {'D8 ✓' if d8['observed_lift'] > d9['observed_lift'] else 'D9'}    │
  │ Incr. Conversions │ {d8['incremental_conv']:>6.1f}        │ {d9['incremental_conv']:>6.1f}        │ {'D8 ✓' if d8['incremental_conv'] > d9['incremental_conv'] else 'D9'}    │
  │ Profit            │ ${d8['profit']:>6.0f}       │ ${d9['profit']:>6.0f}       │ {'D8 ✓' if d8['profit'] > d9['profit'] else 'D9'}    │
  │ Status            │ {'PROFITABLE' if d8['profit'] > 0 else 'UNPROFITABLE':<13} │ {'PROFITABLE' if d9['profit'] > 0 else 'UNPROFITABLE':<13} │         │
  └──────────────────────────────────────────────────────────────┘

THE VERDICT:
  {'✅ HYPOTHESIS CONFIRMED!' if d8['profit'] > 0 and d9['profit'] <= 0 else '⚠️ Results differ from hypothesis'}
  
  {f'Decile 8 is PROFITABLE (${d8["profit"]:.0f}) — these are true Persuadables' if d8['profit'] > 0 else f'Decile 8 is unprofitable (${d8["profit"]:.0f})'}
  {f'Decile 9 is UNPROFITABLE (${d9["profit"]:.0f}) — the "Loyalty Tax" in action' if d9['profit'] <= 0 else f'Decile 9 is marginally profitable (${d9["profit"]:.0f})'}
  
INTERVIEW INSIGHT:
  "My model correctly identified Persuadables, but a calibration issue caused
  'Sure Things' in Decile 9 to be ranked too highly. These customers have the
  highest baseline conversion rate ({d9['control_conv_rate']:.2f}%) but the lowest
  incremental lift ({d9['observed_lift']:.2f}pp) — emailing them is a 'Loyalty Tax'
  that wastes ${abs(d9['profit']):.0f} in this simulation."
""")

---

## 6. Final Summary: Business Recommendations

In [ ]:
# Calculate key metrics for summary
optimal_idx = np.argmax(uplift_curve['profits'])
optimal_pct = uplift_curve['percentages'][optimal_idx] * 100
optimal_profit = uplift_curve['profits'][optimal_idx]
optimal_incr_conv = uplift_curve['incremental_conversions'][optimal_idx]

# At optimal, what does random get?
random_idx_at_optimal = np.argmin(np.abs(random_curve['percentages'] - optimal_pct/100))
random_profit_at_optimal = random_curve['profits'][random_idx_at_optimal]

# Profit improvement over spray & pray
improvement_vs_spray = optimal_profit - spray_and_pray_profit

# Profit improvement over random
improvement_vs_random = optimal_profit - random_profit_at_optimal

print("PHASE 4 SUMMARY: BUSINESS SIMULATION RESULTS")
print("=" * 70)
print(f"""
PARAMETERS:
  Email Cost: ${EMAIL_COST:.2f}
  Profit Margin: ${PROFIT_MARGIN:.2f} per conversion
  Test Set Size: {len(results_df):,} customers

STRATEGY COMPARISON:
  ┌──────────────────────────────────────────────────────────────┐
  │ Strategy          │ % Targeted │ Profit      │ vs Optimal   │
  ├──────────────────────────────────────────────────────────────┤
  │ T-Learner (Best)  │ {optimal_pct:>6.0f}%     │ ${optimal_profit:>9,.0f}  │     —        │
  │ Random @ {optimal_pct:.0f}%      │ {optimal_pct:>6.0f}%     │ ${random_profit_at_optimal:>9,.0f}  │ ${random_profit_at_optimal - optimal_profit:>+,.0f}     │
  │ Spray & Pray      │   100%     │ ${spray_and_pray_profit:>9,.0f}  │ ${spray_and_pray_profit - optimal_profit:>+,.0f}    │
  └──────────────────────────────────────────────────────────────┘

KEY FINDINGS:
  ✅ Optimal targeting: {optimal_pct:.0f}% of customers
  ✅ T-Learner beats Random by ${improvement_vs_random:,.0f} at optimal point
  ✅ T-Learner beats Spray & Pray by ${improvement_vs_spray:,.0f}
  
DECILE 9 VERDICT:
  {'✅ Decile 9 is marginally profitable (${:.0f}) — include in targeting'.format(d9['profit']) if d9['profit'] > 0 else '❌ Decile 9 is UNPROFITABLE (${:.0f}) — the "Loyalty Tax"'.format(d9['profit'])}
  {'   Decile 8 is PROFITABLE (${:.0f}) — true Persuadables identified correctly'.format(d8['profit']) if d8['profit'] > 0 else ''}

INTERVIEW TALKING POINTS:
  1. "By targeting only the top {optimal_pct:.0f}% by uplift, we maximize profit at ${optimal_profit:,.0f}."
  2. "Our model beats random targeting by ${improvement_vs_random:,.0f}."
  3. "I discovered a 'Loyalty Tax': Decile 9 has high conversions but low INCREMENTAL value."
  4. "The T-Learner correctly identifies Persuadables (Decile 8: ${d8['profit']:.0f} profit)."

→ Phase 5: Streamlit deployment to make this interactive
""")

In [ ]:
# Save summary metrics for Phase 5
summary_metrics = {
    'optimal_pct': float(optimal_pct),
    'optimal_profit': float(optimal_profit),
    'spray_and_pray_profit': float(spray_and_pray_profit),
    'random_profit_at_optimal': float(random_profit_at_optimal),
    'improvement_vs_spray': float(improvement_vs_spray),
    'improvement_vs_random': float(improvement_vs_random),
    'd8_profit': float(d8['profit']),
    'd8_lift': float(d8['observed_lift']),
    'd9_profit': float(d9['profit']),
    'd9_lift': float(d9['observed_lift']),
    'd9_control_rate': float(d9['control_conv_rate']),
    'email_cost': EMAIL_COST,
    'profit_margin': PROFIT_MARGIN,
    'test_set_size': len(results_df)
}

import json
with open('../data/simulation_summary.json', 'w') as f:
    json.dump(summary_metrics, f, indent=2)

print("Summary metrics saved to ../data/simulation_summary.json")